In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\上市信息财务信息-成长能力指标.xlsx')

In [3]:
data

,企业编号,标题,日期,营业总收入(元),毛利润(元),归属净利润(元),扣非净利润(元),营业总收入同比增长(元),归属净利润同比增长(元),扣非净利润同比增长(元),营业总收入滚动环比增长(元),归属净利润滚动环比增长(元),扣非净利润滚动环比增长(元)
0,1001,成长能力指标,2018-09-30,1271亿,--,432亿,430亿,1.31%,3.14%,4.19%,2.05%,1.67%,1.67%
1,1001,成长能力指标,2018-06-30,823亿,--,286亿,284亿,-2.13%,1.43%,2.95%,0.92%,1.04%,1.02%
2,1001,成长能力指标,2018-03-31,396亿,--,143亿,143亿,-7.72%,-1.09%,1.86%,-1.97%,-0.29%,0.48%
3,1001,成长能力指标,2017-12-31,1686亿,--,543亿,540亿,4.87%,2.18%,4.56%,1.99%,-0.09%,1.44%
4,1001,成长能力指标,2017-09-30,1255亿,--,419亿,413亿,3.18%,2.97%,4.01%,1.64%,-0.34%,0.34%
5,1001,成长能力指标,2017-06-30,840亿,--,282亿,276亿,1.44%,5.21%,5.40%,0.69%,1.59%,2.10%
6,1001,成长能力指标,2017-03-31,429亿,--,145亿,140亿,0.40%,3.88%,2.37%,0.47%,1.02%,0.63%
7,1001,成长能力指标,2016-12-31,1608亿,--,531亿,517亿,9.72%,4.93%,4.29%,1.23%,-1.89%,-2.24%
8,1001,成长能力指标,2016-09-30,1209亿,--,407亿,397亿,11.32%,9.47%,9.09%,0.52%,1.22%,1.09%
9,1002,成长能力指标,2018-09-30,58.0亿,16.1亿,9.19亿,9.15亿,17.99%,-21.93%,-23.70%,5.20%,-11.87%,-13.78%


In [4]:
#将单位全部统一为以元为单位
def convert(cell):
    pattern1=(r'(-?\d+\.?\d+)亿')
    pattern2=(r'(-?\d+\.?\d+)万亿')
    pattern3=(r'(-?\d+\.?\d+)万')
    if re.findall(pattern1,cell)!=[]:
        cell=float(re.findall(pattern1, cell)[0])*100000000
    elif re.findall(pattern2,cell)!=[]:
        cell=float(re.findall(pattern2, cell)[0])*1000000000000
    elif re.findall(pattern3,cell)!=[]:
        cell=float(re.findall(pattern3, cell)[0])*10000
    else: cell=np.nan
    return cell

In [5]:
data['营业总收入(元)']=data['营业总收入(元)'].map(convert)

In [6]:
data['毛利润(元)']=data['毛利润(元)'].map(convert)

In [7]:
data['归属净利润(元)']=data['归属净利润(元)'].map(convert)

In [8]:
data['扣非净利润(元)']=data['扣非净利润(元)'].map(convert)

In [9]:
data

,企业编号,标题,日期,营业总收入(元),毛利润(元),归属净利润(元),扣非净利润(元),营业总收入同比增长(元),归属净利润同比增长(元),扣非净利润同比增长(元),营业总收入滚动环比增长(元),归属净利润滚动环比增长(元),扣非净利润滚动环比增长(元)
0,1001,成长能力指标,2018-09-30,1.271000e+11,NaN,4.320000e+10,4.300000e+10,1.31%,3.14%,4.19%,2.05%,1.67%,1.67%
1,1001,成长能力指标,2018-06-30,8.230000e+10,NaN,2.860000e+10,2.840000e+10,-2.13%,1.43%,2.95%,0.92%,1.04%,1.02%
2,1001,成长能力指标,2018-03-31,3.960000e+10,NaN,1.430000e+10,1.430000e+10,-7.72%,-1.09%,1.86%,-1.97%,-0.29%,0.48%
3,1001,成长能力指标,2017-12-31,1.686000e+11,NaN,5.430000e+10,5.400000e+10,4.87%,2.18%,4.56%,1.99%,-0.09%,1.44%
4,1001,成长能力指标,2017-09-30,1.255000e+11,NaN,4.190000e+10,4.130000e+10,3.18%,2.97%,4.01%,1.64%,-0.34%,0.34%
5,1001,成长能力指标,2017-06-30,8.400000e+10,NaN,2.820000e+10,2.760000e+10,1.44%,5.21%,5.40%,0.69%,1.59%,2.10%
6,1001,成长能力指标,2017-03-31,4.290000e+10,NaN,1.450000e+10,1.400000e+10,0.40%,3.88%,2.37%,0.47%,1.02%,0.63%
7,1001,成长能力指标,2016-12-31,1.608000e+11,NaN,5.310000e+10,5.170000e+10,9.72%,4.93%,4.29%,1.23%,-1.89%,-2.24%
8,1001,成长能力指标,2016-09-30,1.209000e+11,NaN,4.070000e+10,3.970000e+10,11.32%,9.47%,9.09%,0.52%,1.22%,1.09%
9,1002,成长能力指标,2018-09-30,5.800000e+09,1.610000e+09,9.190000e+08,9.150000e+08,17.99%,-21.93%,-23.70%,5.20%,-11.87%,-13.78%


In [10]:
#将缺失值'--'变为nan
data=data.replace('--',np.nan)

In [11]:
#查看出现较多的时间
data['日期'].value_counts()

2016-12-31    3208
2017-09-30    3204
2017-06-30    3195
2017-03-31    3178
2017-12-31    3133
2018-03-31    3104
2016-09-30    3092
2018-06-30    3065
2018-09-30    3031
2016-06-30     193
2016-03-31     142
2015-12-31     140
2015-09-30      71
2018-12-31      46
2014-12-31      31
2015-06-30       6
2013-12-31       5
15-06-30         4
15-09-30         4
15-12-31         4
16-03-31         4
15-03-31         4
16-06-30         4
14-12-31         3
16-12-31         2
2011-12-31       2
2015-03-31       2
16-09-30         2
14-06-30         2
2012-12-31       2
14-09-30         2
2010-12-31       1
17-03-31         1
Name: 日期, dtype: int64

In [12]:
#补全时间，规整时间的格式
pattern='\d\d\d\d-\d\d-\d\d'
for i in range(len(data)):
    if re.findall(pattern,data.iloc[i,2])==[]:
        data.iloc[i,2]='20'+data.iloc[i,2]

In [13]:
#去除企业编号与日期值同时相同的行，保留第一次出现的行
data.drop_duplicates(subset=['企业编号','日期'],keep='first',inplace=True)

In [14]:
#再次查看出现较多的时间
data['日期'].value_counts()

2016-12-31    2985
2017-09-30    2982
2017-12-31    2980
2018-06-30    2974
2018-09-30    2973
2017-06-30    2972
2018-03-31    2957
2017-03-31    2956
2016-09-30    2873
2016-06-30     156
2015-12-31     128
2016-03-31     112
2015-09-30      60
2018-12-31      45
2014-12-31      33
2015-06-30      10
2015-03-31       6
2013-12-31       5
2014-06-30       2
2014-09-30       2
2012-12-31       2
2011-12-31       2
2010-12-31       1
Name: 日期, dtype: int64

In [15]:
#将数量少于2800的日期全部丢弃
timelist=list(['2016-12-31','2017-09-30','2017-12-31','2018-06-30','2018-09-30','2017-06-30','2018-03-31','2017-03-31','2016-09-30'])
data=data[data['日期'].isin(timelist)]
data

,企业编号,标题,日期,营业总收入(元),毛利润(元),归属净利润(元),扣非净利润(元),营业总收入同比增长(元),归属净利润同比增长(元),扣非净利润同比增长(元),营业总收入滚动环比增长(元),归属净利润滚动环比增长(元),扣非净利润滚动环比增长(元)
0,1001,成长能力指标,2018-09-30,1.271000e+11,NaN,4.320000e+10,4.300000e+10,1.31%,3.14%,4.19%,2.05%,1.67%,1.67%
1,1001,成长能力指标,2018-06-30,8.230000e+10,NaN,2.860000e+10,2.840000e+10,-2.13%,1.43%,2.95%,0.92%,1.04%,1.02%
2,1001,成长能力指标,2018-03-31,3.960000e+10,NaN,1.430000e+10,1.430000e+10,-7.72%,-1.09%,1.86%,-1.97%,-0.29%,0.48%
3,1001,成长能力指标,2017-12-31,1.686000e+11,NaN,5.430000e+10,5.400000e+10,4.87%,2.18%,4.56%,1.99%,-0.09%,1.44%
4,1001,成长能力指标,2017-09-30,1.255000e+11,NaN,4.190000e+10,4.130000e+10,3.18%,2.97%,4.01%,1.64%,-0.34%,0.34%
5,1001,成长能力指标,2017-06-30,8.400000e+10,NaN,2.820000e+10,2.760000e+10,1.44%,5.21%,5.40%,0.69%,1.59%,2.10%
6,1001,成长能力指标,2017-03-31,4.290000e+10,NaN,1.450000e+10,1.400000e+10,0.40%,3.88%,2.37%,0.47%,1.02%,0.63%
7,1001,成长能力指标,2016-12-31,1.608000e+11,NaN,5.310000e+10,5.170000e+10,9.72%,4.93%,4.29%,1.23%,-1.89%,-2.24%
8,1001,成长能力指标,2016-09-30,1.209000e+11,NaN,4.070000e+10,3.970000e+10,11.32%,9.47%,9.09%,0.52%,1.22%,1.09%
9,1002,成长能力指标,2018-09-30,5.800000e+09,1.610000e+09,9.190000e+08,9.150000e+08,17.99%,-21.93%,-23.70%,5.20%,-11.87%,-13.78%


In [16]:
data.sort_values(by=['企业编号','日期'],inplace=True)

In [17]:
#查看排序结果是否达到目标效果
data

,企业编号,标题,日期,营业总收入(元),毛利润(元),归属净利润(元),扣非净利润(元),营业总收入同比增长(元),归属净利润同比增长(元),扣非净利润同比增长(元),营业总收入滚动环比增长(元),归属净利润滚动环比增长(元),扣非净利润滚动环比增长(元)
8,1001,成长能力指标,2016-09-30,1.209000e+11,NaN,4.070000e+10,3.970000e+10,11.32%,9.47%,9.09%,0.52%,1.22%,1.09%
7,1001,成长能力指标,2016-12-31,1.608000e+11,NaN,5.310000e+10,5.170000e+10,9.72%,4.93%,4.29%,1.23%,-1.89%,-2.24%
6,1001,成长能力指标,2017-03-31,4.290000e+10,NaN,1.450000e+10,1.400000e+10,0.40%,3.88%,2.37%,0.47%,1.02%,0.63%
5,1001,成长能力指标,2017-06-30,8.400000e+10,NaN,2.820000e+10,2.760000e+10,1.44%,5.21%,5.40%,0.69%,1.59%,2.10%
4,1001,成长能力指标,2017-09-30,1.255000e+11,NaN,4.190000e+10,4.130000e+10,3.18%,2.97%,4.01%,1.64%,-0.34%,0.34%
3,1001,成长能力指标,2017-12-31,1.686000e+11,NaN,5.430000e+10,5.400000e+10,4.87%,2.18%,4.56%,1.99%,-0.09%,1.44%
2,1001,成长能力指标,2018-03-31,3.960000e+10,NaN,1.430000e+10,1.430000e+10,-7.72%,-1.09%,1.86%,-1.97%,-0.29%,0.48%
1,1001,成长能力指标,2018-06-30,8.230000e+10,NaN,2.860000e+10,2.840000e+10,-2.13%,1.43%,2.95%,0.92%,1.04%,1.02%
0,1001,成长能力指标,2018-09-30,1.271000e+11,NaN,4.320000e+10,4.300000e+10,1.31%,3.14%,4.19%,2.05%,1.67%,1.67%
17,1002,成长能力指标,2016-09-30,4.530000e+09,1.770000e+09,1.060000e+09,1.040000e+09,8.06%,14.11%,13.71%,1.58%,4.05%,3.70%


In [18]:
data=data.set_index(['企业编号','日期'])

In [19]:
data=data.unstack().reset_index().set_index('企业编号').T.drop('标题')
data=data.reindex(columns=list(range(1001,4001)))
data.to_pickle('上市信息财务信息-成长能力指标.pickle')
data

企业编号                            1001      1002       1003      1004  \
               日期                                                     
营业总收入(元)       2016-09-30  1.209e+11  4.53e+09   1.05e+10  1.72e+09   
               2016-12-31  1.608e+11  6.17e+09    1.6e+10  2.35e+09   
               2017-03-31   4.29e+10  1.58e+09   3.84e+09  6.02e+08   
               2017-06-30    8.4e+10  3.21e+09   8.15e+09  1.25e+09   
               2017-09-30  1.255e+11  4.92e+09    1.3e+10  1.99e+09   
               2017-12-31  1.686e+11  6.76e+09   1.83e+10  2.77e+09   
               2018-03-31   3.96e+10  1.72e+09   2.46e+09  7.78e+08   
               2018-06-30   8.23e+10  3.72e+09   5.79e+09  1.55e+09   
               2018-09-30  1.271e+11   5.8e+09   9.24e+09  2.34e+09   
毛利润(元)         2016-09-30        NaN  1.77e+09    1.4e+09  8.77e+08   
               2016-12-31        NaN  2.38e+09   1.95e+09  1.13e+09   
               2017-03-31        NaN  5.99e+08   3.65e+08  2.66e+08   
               2017-06-30        NaN  1.24e+09    7.7e+08  5.55e+08   
               2017-09-30        NaN  1.88e+09   1.24e+09  8.63e+08   
               2017-12-31        NaN  2.65e+09   1.74e+09  1.17e+09   
               2018-03-31        NaN  6.68e+08    3.5e+08  3.34e+08   
               2018-06-30        NaN  1.16e+09    6.9e+08  6.82e+08   
               2018-09-30        NaN  1.61e+09   1.01e+09  1.03e+09   
归属净利润(元)       2016-09-30   4.07e+10  1.06e+09   2.27e+08  5.49e+08   
               2016-12-31   5.31e+10  1.39e+09   2.19e+08  6.86e+08   
               2017-03-31   1.45e+10  4.08e+08  7.031e+07  1.59e+08   
               2017-06-30   2.82e+10  7.81e+08   1.35e+08  2.95e+08   
               2017-09-30   4.19e+10  1.18e+09   1.15e+08  4.77e+08   
               2017-12-31   5.43e+10   1.6e+09   2.01e+08  6.35e+08   
               2018-03-31   1.43e+10  4.33e+08   1.29e+08  2.02e+08   
               2018-06-30   2.86e+10  7.03e+08   2.91e+08  4.04e+08   
               2018-09-30   4.32e+10  9.19e+08   4.14e+08   5.9e+08   
扣非净利润(元)       2016-09-30   3.97e+10  1.04e+09    2.2e+08  5.48e+08   
               2016-12-31   5.17e+10  1.39e+09   2.05e+08  6.87e+08   
               2017-03-31    1.4e+10  3.81e+08  6.583e+07  1.57e+08   
...                              ...       ...        ...       ...   
扣非净利润同比增长(元)   2018-03-31      1.86%    13.48%     75.14%    28.26%   
               2018-06-30      2.95%   -10.18%    102.83%    39.97%   
               2018-09-30      4.19%   -23.70%    245.30%    31.78%   
营业总收入滚动环比增长(元) 2016-09-30      0.52%     1.58%     -0.89%     0.79%   
               2016-12-31      1.23%     3.28%      1.27%     0.42%   
               2017-03-31      0.47%     2.03%      3.28%     1.32%   
               2017-06-30      0.69%     1.10%      4.44%     3.32%   
               2017-09-30      1.64%     2.98%      7.30%     6.46%   
               2017-12-31      1.99%     3.22%     -1.28%     5.67%   
               2018-03-31     -1.97%     2.07%     -7.53%     6.34%   
               2018-06-30      0.92%     5.32%     -5.78%     4.14%   
               2018-09-30      2.05%     5.20%     -8.76%     1.54%   
归属净利润滚动环比增长(元) 2016-09-30      1.22%     4.05%    -27.60%     1.51%   
               2016-12-31     -1.89%     0.74%     -8.67%    -3.41%   
               2017-03-31      1.02%     4.33%    -23.30%    -2.58%   
               2017-06-30      1.59%     1.45%    -21.83%    -9.70%   
               2017-09-30     -0.34%     2.58%    -18.45%     1.81%   
               2017-12-31     -0.09%     5.43%     87.56%     3.26%   
               2018-03-31     -0.29%     1.57%     29.49%     6.78%   
               2018-06-30      1.04%    -6.34%     37.44%     9.72%   
               2018-09-30      1.67%   -11.87%     39.75%     0.51%   
扣非净利润滚动环比增长(元) 2016-09-30      1.09%     3.70%    -21.72%     2.26%   
               2016-12-31     -2.24%     2.16%     14.21%    -2.65%   
               2017-03-3